In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# train set, test set and validation set paths
Dir = "../input/chest-xray-pneumonia/chest_xray"
train_dir = Dir + "/train"
test_dir = Dir + "/test"
val_dir = Dir + "/val"

Categories = ['NORMAL', 'PNEUMONIA']

In [ ]:
# Normal and Pneumonia paths
normal_train_dir = train_dir + "NORMAL"
pneumonia_train_dir = train_dir + "PNEUMONIA"

In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

In [ ]:
IMG_SIZE = 150
training_data = []

In [ ]:
def create_training_data(D):
    for category in Categories:
        path = os.path.join(D, category)
        class_num = Categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data(train_dir)
create_training_data(test_dir)
create_training_data(val_dir)

In [ ]:
import random
random.shuffle(training_data)
train = pd.DataFrame(training_data,columns=['Image', 'Class'])

In [ ]:
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3) #3 cause RGB
y = np.array(y)

X = X/255.0

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, SeparableConv2D, BatchNormalization

In [ ]:
model = Sequential()

#Input
model.add(SeparableConv2D(64, (3,3), input_shape = X.shape[1:], data_format="channels_last"))
model.add(Activation("relu"))
model.add(SeparableConv2D(64, (3,3), data_format="channels_last"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#First
model.add(SeparableConv2D(128, (3,3), data_format="channels_last"))
model.add(Activation("relu"))
model.add(SeparableConv2D(128, (3,3), data_format="channels_last"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#Second
model.add(SeparableConv2D(256, (3,3), data_format="channels_last"))
model.add(Activation("relu"))
model.add(SeparableConv2D(256, (3,3), data_format="channels_last"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#Third
model.add(SeparableConv2D(512, (3,3), data_format="channels_last"))
model.add(Activation("relu"))
model.add(SeparableConv2D(512, (3,3), data_format="channels_last"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) # converts our 3D feature maps to 1D feature vectors
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.7))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation("softmax"))

model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

In [ ]:
model.build()
model.summary()

In [ ]:
model.fit(X, y, batch_size=24, epochs=10, validation_split=0.2)

In [ ]:
model.fit(X, y, batch_size=BATCH_SIZE, epochs=10, validation_split=0.25)

In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
model.save("model.h5")

In [ ]:
model1 = tf.keras.models.load_model('model.h5')

In [ ]:
#model1.summary()